<a href="https://colab.research.google.com/github/SifolDifol/HomeWorkForCourse/blob/pandas-%2F-numpy/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%E2%84%969.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 37. Word Embeddings

## Домашнее задание

**Цель**: Разработать классификатор для определения спам/не с пам сообщений с использованием различных методов предобработки текста и векторизации.

**Задания**:
1. Подготовка данных:
    Загрузите датасет SMS сообщений,размеченных как спамилинеспам. https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

2. Предобработка текста: Реализуйте лемматизацию или стемминг для текстов сообщений.
3. Векторизация текста: Примените мешок слов(BagofWords)и TF-IDF для векторизации текста.
4. Моделирование: Постройте модели для классификации сообщений как спам или неспам(модели на ваш выбор).
5. Оцените качество моделей с различными комбинациями предобработкии векторизации(сравните ROC-AUC,F1-score,accuracyит.д.).

In [33]:
import pandas as pd
import matplotlib.pyplot as plt

# Устанавливаем стиль визуализаций в matplotlib
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

# Загрузка данных с указанием кодировки
df = pd.read_csv('drive/MyDrive/Colab Notebooks/dataForWork/spam.csv', encoding='latin-1')

df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [34]:
df = df[['v1', 'v2']]
df.columns = ['label', 'message']
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [35]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [36]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess_text(text, method='lemmatization'):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    words = nltk.wordpunct_tokenize(text)

    if method == 'lemmatization':
        return ' '.join([lemmatizer.lemmatize(word) for word in words])

    return ' '.join([stemmer.stem(word) for word in words])


df['preprocess_message'] = df['message'].apply(preprocess_text)
df.head()

,label,message,preprocess_message
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i don t think he go to usf he life around ...


In [37]:
bow_vectorizer = CountVectorizer()
X_bow = bow_vectorizer.fit_transform(df['preprocess_message'])

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['preprocess_message'])

In [38]:
X_bow.toarray()[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

y = df['label'].map({'ham': 0, 'spam': 1})

X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.99      0.85      0.92       150

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [41]:
print(roc_auc_score(y_test, y_pred))

0.9261485319516408
